# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Prompt Engineering Best Practices: Text Expansion </center>
***



Text expansion is the task of taking a shorter piece of text, such as a set of instructions or a list of topics, and having the large language model generate a longer piece of text, such as an email or an essay about some topic.

There are some great uses of this, such as if you use a large language model as a brainstorming partner. However there are also some problematic use cases of this, such as if someone were to use it, they generate a large amount of spam. 

In this article, we'll go through an example of how you can use a language model to generate a personalized email based on some information. The email is self-proclaimed to be from an AI bot which  is very important.

 We're also going to use another one of the model's input parameters called "temperature" which  allows you to vary the kind of degree of exploration and variety in the kind of model's responses. So let's get into it!
 
 
 #### <a id="top"></a>

# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
<ul>
    <li><a href="#1" target="_self" rel=" noreferrer nofollow">1. Setting Up the Working Environment </a> </li>
    <li><a href="#2" target="_self" rel=" noreferrer nofollow">2. Generated Customer Email Responses </a></li>
    <li><a href="#3" target="_self" rel=" noreferrer nofollow">3. Changing the Tempurature </a></li>     
</ul>
</div>

***

 
 

<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Setting Up the Working Environment </b></div>

Let's start with importing OpenAI and loading the environment variables


In [ ]:
import os
import utils
import panel as pn
from openai import OpenAI
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI

# ==============================
# 🔑 Load environment variables
# ==============================
env_values = dotenv_values(".env")

openai_api_key = env_values.get("OPENAI_API_KEY")
openai_api_base = env_values.get("OPENAI_API_BASE")
openai_api_name = env_values.get("OPENAI_API_NAME")

# Initialize Clients
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base
)

# Mock moderation for OpenRouter compatibility
if openai_api_base and "openrouter.ai" in openai_api_base:
    class MockResult(dict):
        def __getattr__(self, name): return self.get(name)
    class MockModeration(dict):
        def __init__(self):
            res = MockResult(flagged=False, categories={}, category_scores={})
            super().__init__(results=[res])
        @property
        def results(self): return self["results"]
    client.moderations.create = lambda *args, **kwargs: MockModeration()

chat = ChatOpenAI(
    model_name=openai_api_name, 
    openai_api_key=openai_api_key, 
    openai_api_base=openai_api_base,
    temperature=0.7
)
pn.extension()



C:\Users\Administrator\AppData\Local\Temp\ipykernel_13924\312141790.py:41: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


Then we will define the get_completion function. The get_completion function is a Python function designed to interact with a chatbot model, likely provided by a service such as OpenAI's ChatGPT. 
It takes a user prompt and a model specification as inputs, sends the prompt to the chatbot model through an API, and returns the generated completion. The function abstracts away the complexity of interacting with the API, providing a simple interface for generating chatbot responses based on user input.

In [ ]:
# ==============================
# 🔧 Helper Functions
# ==============================
def get_completion(prompt, model=openai_api_name, temperature=0, max_tokens=500):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model=openai_api_name, temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Generated Customer Email Responses </b></div>


Let's write a custom email response to a customer review. The customer review and the sentiment are given and we're going to generate a custom response.

In [3]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn't look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

Now we're going to use the language model to generate a custom email to a customer based on a customer review and the sentiment of the review. We have already seen how to extract the sentiment using prompt engineering in the [Prompt Engineering for Instruction-Tuned LLM: Textual Inference & Sentiment Analysis](https://www.kaggle.com/code/youssef19/textual-inference-sentiment-analysis) notebook. 
Let's  start customizing the reply based on the sentiment and the review we got from the customer. Here is the prompt and the instruction we will give to the LLM.

In [4]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Subject: Thank You for Your Feedback

Dear Valued Customer,

Thank you for taking the time to share your thoughts with us. We sincerely apologize for the experience you had with the pricing changes and the quality concerns regarding the 17 piece system. We understand how frustrating it can be when expectations are not met, especially after relying on our products for your culinary needs.

Your feedback is invaluable, and we encourage you to reach out to our customer service team for any assistance you may need. We are here to help and want to ensure that you have a positive experience with our products.

Thank you once again for your insights.

Best regards,

AI customer agent


<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Changing the Tempurature </b></div>



Next, we're going to use a parameter of the language model called **temperature** that will allow us to change the kind of variety of the model's responses. You can think of **temperature** as the degree of exploration or kind of randomness of the model. 

Let's take this particular phrase, "my favorite food is", the most likely next word that the model predicts is "pizza", and the next to most likely it suggests are "sushi" and "tacos". At a temperature of zero, the model will always choose the most likely next word, which in this case is "pizza", and at a higher **temperature**, it will also choose one of the less likely words, and even at an even higher **temperature**, it might even choose "tacos", which only kind of has a 5% chance of being chosen. 

You can imagine that kind of as the model continues this final response, so my favorite food is pizza, and it kind of continues to generate more words, this response will kind of diverge from the response, the first response, which is my favorite food is tacos. These two responses will become more and more different. 

In general, when building applications where you want a predictable response, I would recommend using **temperature** = zero to build a system that is reliable and predictable, you should go with this. If you're trying to use the model in a more creative way, where you might want a kind of wider variety of different outputs, you might want to use a higher temperature. 

Let's take the same prompt that we just used before and use a higher temperature. So in the "**get_completion**" function that we've been using throughout this series of articles, we have specified the model and the temperature, but we have set them to default. 
We will set the **temperature** to 0.7. When we used the **temperature** =  zero, every time you execute the same prompt, you should expect the same completion. Whereas with **temperature** 0.7, you'll get a different output every time.

In [5]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

Subject: Thank You for Your Feedback

Dear Valued Customer,

Thank you for taking the time to share your thoughts regarding your recent experience with our products. We sincerely apologize for the price changes you encountered and any disappointment this may have caused. Your feedback about the quality of the system and motor noise is important to us, and we take such concerns seriously.

If you have further issues or need assistance, please feel free to reach out to our customer service team. We are here to help and ensure you have a better experience with our products.

Thank you again for your feedback.

Best regards,

AI customer agent


Here is the generated email, and as you can see, it's different from the email that we received previously. If you execute it again it will generate a different email again.